In [1]:
import argparse
import numpy as np
from pprint import pprint
from tqdm import tqdm
from PIL import Image
import matplotlib.pyplot as plt
import torch.optim as optim
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import grad
from torchvision import datasets, transforms
import os
from blackbox_utils import label_to_onehot, cross_entropy_for_onehot, pixelwise_euclidean_distance
import matplotlib.pyplot as plt
import matplotlib  # 关键：显式
from torch.utils.data import Dataset, DataLoader, Subset
# 指定使用文泉驿微米黑（系统已识别的字体名称）# 获取 matplotlib 缓存目录
cache_dir = matplotlib.get_cachedir()  
# 字体缓存文件名（不同版本可能为 fontlist-v330.json、fontlist.json 等）
font_cache = os.path.join(cache_dir, "fontlist.json")  

# 删除缓存文件
if os.path.exists(font_cache):
    os.remove(font_cache)
    print("✅ 已删除字体缓存，重启内核后生效！")
else:
    print("⚠️ 未找到字体缓存文件，可尝试重启内核。")
plt.rcParams['font.sans-serif'] = ['WenQuanYi Micro Hei']  
# 解决负号显示为方块的问题
plt.rcParams['axes.unicode_minus'] = False  
##########sudo apt update && sudo apt install fonts-wqy-microhei -y  

/environment/miniconda3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


⚠️ 未找到字体缓存文件，可尝试重启内核。


In [2]:
# ################################################
# ## 轻易不要运行这个代码

# import shutil

# # 配置路径
# source_root = "/home/featurize/data/University-Release/train/drone"
# target_dir = "/home/featurize/data/blackbox_data"

# # 确保目标文件夹存在
# if not os.path.exists(target_dir):
#     os.makedirs(target_dir)
#     print(f"创建目标文件夹: {target_dir}")

# # 定义支持的图片扩展名（可根据实际情况补充）
# IMAGE_EXTENSIONS = ('.jpg', '.jpeg', '.png', '.bmp', '.gif', '.tiff')

# # 遍历所有子文件夹，提取图片
# copied_count = 0
# for root, dirs, files in os.walk(source_root):
#     for filename in files:
#         # 检查文件扩展名是否为图片
#         if filename.lower().endswith(IMAGE_EXTENSIONS):
#             source_path = os.path.join(root, filename)
#             # 处理重名：自动添加序号（如 photo.jpg → photo_1.jpg）
#             target_path = os.path.join(target_dir, filename)
#             if os.path.exists(target_path):
#                 base_name, ext = os.path.splitext(filename)
#                 counter = 1
#                 while True:
#                     new_filename = f"{base_name}_{counter}{ext}"
#                     target_path = os.path.join(target_dir, new_filename)
#                     if not os.path.exists(target_path):
#                         break
#                     counter += 1

#             # （可选）验证图片完整性，避免复制损坏文件
#             try:
#                 with Image.open(source_path) as img:
#                     img.verify()  # 验证图片是否可正常打开
#             except (IOError, SyntaxError) as e:
#                 print(f"跳过无效图片: {source_path}（错误: {e}）")
#                 continue

#             # 复制文件（保留元数据）
#             shutil.copy2(source_path, target_path)
#             ########### print(f"已复制: {source_path} → {target_path}")
#             copied_count += 1

# print(f"\n任务完成！共复制 {copied_count} 张图片到 {target_dir}")

In [3]:
# ------------------- 配置参数 -------------------
batchsize = 1
setpoint = 4####实际为3
h, w = 384, 384
pad = 10
num_images = 3000  # 需要选取的照片数量
target_dir = '/home/featurize/data/blackbox_data'  # 目标图片路径
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tt = transforms.ToTensor()#转发为张量
tp = transforms.ToPILImage()#转换为图片
print(f"run in {device}")

run in cuda


In [4]:
# ################################################
# ## 轻易不要运行这个代码

# import shutil

# # 配置路径
# source_root = "/home/featurize/data/University-Release/train/drone"
# target_dir = "/home/featurize/data/blackbox_data"

# # 确保目标文件夹存在
# if not os.path.exists(target_dir):
#     os.makedirs(target_dir)
#     print(f"创建目标文件夹: {target_dir}")

# # 定义支持的图片扩展名（可根据实际情况补充）
# IMAGE_EXTENSIONS = ('.jpg', '.jpeg', '.png', '.bmp', '.gif', '.tiff')

# # 遍历所有子文件夹，提取图片
# copied_count = 0
# for root, dirs, files in os.walk(source_root):
#     for filename in files:
#         # 检查文件扩展名是否为图片
#         if filename.lower().endswith(IMAGE_EXTENSIONS):
#             source_path = os.path.join(root, filename)
#             # 处理重名：自动添加序号（如 photo.jpg → photo_1.jpg）
#             target_path = os.path.join(target_dir, filename)
#             if os.path.exists(target_path):
#                 base_name, ext = os.path.splitext(filename)
#                 counter = 1
#                 while True:
#                     new_filename = f"{base_name}_{counter}{ext}"
#                     target_path = os.path.join(target_dir, new_filename)
#                     if not os.path.exists(target_path):
#                         break
#                     counter += 1

#             # （可选）验证图片完整性，避免复制损坏文件
#             try:
#                 with Image.open(source_path) as img:
#                     img.verify()  # 验证图片是否可正常打开
#             except (IOError, SyntaxError) as e:
#                 print(f"跳过无效图片: {source_path}（错误: {e}）")
#                 continue

#             # 复制文件（保留元数据）
#             shutil.copy2(source_path, target_path)
#             ########### print(f"已复制: {source_path} → {target_path}")
#             copied_count += 1

# print(f"\n任务完成！共复制 {copied_count} 张图片到 {target_dir}")

In [5]:
# 自定义数据集
class CustomImageDataset(Dataset):
    def __init__(self, image_dir, transform=None):
        self.image_dir = image_dir
        self.transform = transform
        # 只保留图片文件
        self.image_paths = [
            os.path.join(image_dir, fname) 
            for fname in os.listdir(image_dir) 
            if fname.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp'))
            and not fname.startswith('.')
        ]

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        # 加载并返回 PIL 图像
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert('RGB')
        
        # 应用预处理转换
        if self.transform:
            image = self.transform(image)
            
        return image, 0  # 返回预处理后的图像和标签 0

# 自定义 collate_fn
def custom_collate(batch):
    return batch[0]  # batch_size=1 时直接返回单个样本

# 定义图像预处理转换
transform = transforms.Compose([
    transforms.Resize((384, 384)),  # 调整图像大小为384×384
    transforms.ToTensor()         # 转换为张量
    # transforms.Normalize(           # 归一化处理
    #     mean=[0.485, 0.456, 0.406],  # ImageNet数据集的均值
    #     std=[0.229, 0.224, 0.225]    # ImageNet数据集的标准差
    
])

# 创建数据集和数据加载器
dataset = CustomImageDataset(target_dir, transform=transform)
if len(dataset.image_paths) > 3000:
    dataset.image_paths = dataset.image_paths[:3000]
    print(f"数据集已限制为 {len(dataset.image_paths)} 张照片")
else:
    print(f"数据集共有 {len(dataset.image_paths)} 张照片，全部使用")


# 创建数据加载器（指定自定义 collate_fn）
dataloader = DataLoader(
    dataset,
    batch_size=1,  
    shuffle=False,
    num_workers=0,
    pin_memory=True,
    collate_fn=custom_collate  # 关键修改
)

# 显示第三张照片
image_tensor, label = list(dataloader)[2]  # 获取第 3 张（索引 2）
image_pil = transforms.ToPILImage()(image_tensor)  # 转换回PIL图像用于显示

plt.figure(figsize=(8, 8))
plt.imshow(image_pil)
# plt.title(f"第 3 张照片 (标签: {label})")
plt.axis('off')
plt.show()

NameError: name 'target_dir' is not defined

In [ ]:
from model_black_attack import ft_net
pretrained_path = '/home/featurize/work/RK_net/RK-Net-main/model/pretrain/net_359.pth'  # 绝对路径
# RK-Net/RK-Net-main/model/pretrain/net_359.pth
net = ft_net(751, sp = setpoint).to(device)

# 加载预训练权重（strict=False允许部分参数不匹配，适用于迁移学习）

pretrained_state = torch.load(pretrained_path, map_location=torch.device('cpu'))
net.load_state_dict(pretrained_state, strict=False)
print("Loaded pretrained model from:", pretrained_path)

| 索引 | 模块名称                          | 说明                          |     
|------|-----------------------------------|-----------------------------|
| 0    | `model_ft.conv1`                  | 第1层：输出[1, 64, 192, 192] |
| 1    | `model_ft.bn1`                    | 第2层：批归一化层             |
| 2    | `model_ft.relu`                   | 第3层：ReLU激活函数           |
| 3    | `self.usam_1`                     | 第4层：输出[1, 64, 192, 192]  |
| 4    | `model_ft.maxpool`                | 第5层：最大池化层             |
| 5    | `model_ft.layer1`                 | 第6层：输出[1, 256, 96, 96]  |
| 6    | `self.usam_2`                     | 第7层：输出[1, 256, 96, 96]  |
| 7    | `model_ft.layer2`                 | 第8层：输出[1, 512, 48, 48]  |
| 8    | `model_ft.layer3`                 | 第9层：输出[1, 1024, 24, 24] |
| 9    | `model_ft.layer4`                 | 第10层：输出[1, 2048, 12, 12]|
| 10   | `model_ft.avgpool2`               | 第11层：全局平均池化层        |
| 11   | `lambda x: x.view(x.size(0), x.size(1))` | 第12层：特征展平操作（将空间维度展平为特征向量） |


In [ ]:
from model_black_attack import blackbox_attack0, blackbox_attack5, blackbox_attack7, blackbox_attack8, blackbox_attack9, blackbox_attack3
if setpoint == 6:
    blackbox_net = blackbox_attack5(num_classes=751).to(device)##[256, 96, 96]
if setpoint == 8:
    blackbox_net = blackbox_attack7(num_classes=751).to(device)
if setpoint == 1:
    blackbox_net = blackbox_attack0(num_classes=751).to(device)
if setpoint == 4:
    blackbox_net = blackbox_attack3(num_classes=751).to(device)#和1一样
if setpoint == 7:
    blackbox_net = blackbox_attack5(num_classes=751).to(device)###[256, 96, 96
if setpoint == 9:
    blackbox_net = blackbox_attack8(num_classes=751).to(device)###1024, 24, 24
if setpoint == 10:
    blackbox_net = blackbox_attack9(num_classes=751).to(device)###2948, 12, 12
print(blackbox_net)
weight_path = "w3_attack.pth"  # 明确权重路径
if os.path.isfile(weight_path):
    # 加载权重（自动适配设备，避免强制CPU）
    pretrained_state = torch.load(weight_path, map_location=device)  
    # 加载到模型（strict=False 兼容结构差异，根据需求调整）
    blackbox_net.load_state_dict(pretrained_state, strict=False)  

In [ ]:
# 获取最后一张照片（索引为 len(dataset)-1 ）
last_idx = len(dataset) - 19
image_attack, label_attack = dataset[last_idx]  

# 转换为 PIL 图像并显示（与之前逻辑一致）
image_show = transforms.ToPILImage()(image_attack)  
print(image_show.size)

plt.figure(figsize=(8, 8))
plt.imshow(image_show)
# plt.title(f"原图")
plt.axis('off')
plt.show()

with torch.no_grad():
        image_show = tt(image_show).unsqueeze(0).to(device)  # 添加 batch_size=1 的维度
        show_interdata = net.f(image_show)    # net 需提前定义（模型实例）
        show_interdata = show_interdata
        
with torch.no_grad():
    show_interdata = show_interdata.to(device)
        # 前向传播
    show_outputs = blackbox_net(show_interdata)
plt.figure(figsize=(8, 8))
plt.imshow(tp(show_outputs[0].cpu()))
# plt.title(f"结果展示")
plt.axis('off')
plt.show()

In [ ]:
# tt = transforms.ToTensor()#转发为张量
# tp = transforms.ToPILImage()#转换为图片
# def image2prob(image_dummy):
#     image_dummy_tensor = tt(image_dummy)
#     # 计算三通道总和并对每个通道分别归一化
#     total_sum = image_dummy_tensor.sum(dim=(1, 2))
#     image_dummy_norm = image_dummy_tensor / total_sum.view(-1, 1, 1)
#     dummy_prob = F.softmax(image_dummy_norm.view(-1), dim=0)
#     return dummy_prob

# # attack_image = dummy_data.detach().clone()
# #print(type(gt_data))
# # image_dummy = history[9]
# attack_prob = image2prob(tp(show_outputs[0]))
# ture_prob = image2prob(tp(image_show[0]))
# kl_div = F.kl_div(ture_prob.log(), attack_prob, reduction='batchmean')
# print(abs(kl_div))

In [ ]:
def ssim(img1, img2, window_size=11, size_average=True):
    """
    计算两张图片的结构相似性指数(SSIM)
    
    参数:
        img1: 张量，形状为[batch, channels, height, width]
        img2: 张量，形状为[batch, channels, height, width]
        window_size: 高斯窗口大小
        size_average: 是否对批次和通道取平均
    
    返回:
        ssim值，范围[-1, 1]，越接近1表示越相似
    """
    # 确保输入是浮点型（此时 img1、img2 已是张量，可直接操作）
    img1 = img1.float()
    img2 = img2.float()
    
    # 获取输入尺寸
    batch, channels, height, width = img1.size()
    
    # 修正创建高斯窗口的方式，使用张量进行计算
    x = torch.arange(window_size).float()  # 创建张量来承载x的取值，转换为浮点型
    numerator = -(x - window_size // 2) ** 2
    denominator = float(2 * (window_size // 4) ** 2)
    gauss = torch.exp(numerator / denominator)
    gauss = gauss / gauss.sum()
    window_1d = gauss.unsqueeze(1)
    window_2d = window_1d.mm(window_1d.t()).float().unsqueeze(0).unsqueeze(0)
    window = window_2d.expand(channels, 1, window_size, window_size).contiguous()
    
    # 将窗口移动到与输入相同的设备
    if img1.is_cuda:
        window = window.cuda(img1.get_device())
    window = window.to(img1.device)
    
    # 计算均值
    mu1 = F.conv2d(img1, window, padding=window_size//2, groups=channels)
    mu2 = F.conv2d(img2, window, padding=window_size//2, groups=channels)
    
    # 计算方差和协方差
    mu1_sq = mu1.pow(2)
    mu2_sq = mu2.pow(2)
    mu1_mu2 = mu1 * mu2
    
    sigma1_sq = F.conv2d(img1*img1, window, padding=window_size//2, groups=channels) - mu1_sq
    sigma2_sq = F.conv2d(img2*img2, window, padding=window_size//2, groups=channels) - mu2_sq
    sigma12 = F.conv2d(img1*img2, window, padding=window_size//2, groups=channels) - mu1_mu2
    
    # SSIM公式
    C1 = 0.01**2
    C2 = 0.03**2
    
    ssim_map = ((2*mu1_mu2 + C1) * (2*sigma12 + C2)) / ((mu1_sq + mu2_sq + C1) * (sigma1_sq + sigma2_sq + C2))
    
    if size_average:
        return ssim_map.mean()
    else:
        return ssim_map.mean(1).mean(1).mean(1)

# similarity = ssim(tp(show_outputs[0]), tp(image_show[0]))
similarity = ssim(show_outputs, image_show)
print(f"相似度为{similarity}")